In [58]:
import os 
import json
import torch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
from typing import Callable,List

In [3]:
BASE_DIR = Path(os.getcwd()).parent.parent
%run ../paths.ipynb

download paths


In [25]:
load_dotenv(override=True,dotenv_path=PATH_ENV)
openai_key_pinecone = os.getenv("token_pinecone")
pc = Pinecone(api_key = openai_key_pinecone)

In [ ]:
def create_embeddings(tokens,model="all-MiniLM-L6-v2"): 
    """
    Generate embeddings whit hugginface 
    """
    ### creating data for pinecone 
    model = SentenceTransformer(model,device='cpu')
    embeddings = list(map(lambda x: model.encode(x).tolist(),tokens))
    embddings_list = [(str(i),embeddings[i],{"text":tokens[i]}) for i in range(len(embeddings))]    
    
    return embddings_list

def insert_embeddings(
        name_tokenization: str, 
        name_index: str, 
        namespace: str,
        embeding_function: Callable[[List[str]], List[float]],
        model: str = "all-MiniLM-L6-v2",
): 
    """
    Create namespace with the embeddings for pinecone, with all-MiniLM-L6-v2 transformer
    Only search tokens for the thesis document
    """    

    file_tokens_thesis = open(TOKENS_THESIS,"r",encoding="utf-8")
    tokens_thesis = json.load(file_tokens_thesis)
    tokens_thesis = tokens_thesis[name_tokenization]["values"]
    
    
    embddings_list = embeding_function(tokens_thesis,model)

    indexes  = pc.list_indexes()
    name_indixes = [index["name"] for index  in indexes]
    assert name_index in name_indixes, f"don't exist {name_index} in pinecone"

    idx = pc.Index(name_index)
    idx.upsert(embddings_list,namespace=namespace)

    return idx



In [ ]:
insert_embeddings(
        name_tokenization = "first_tokenization",
        name_index = "tweens",
        namespace = "first_tokenization_thesis",
        embeding_function = create_embeddings,
        model = "all-MiniLM-L6-v2"
)

In [62]:
idx_tweens = pc.Index("tweens")
namespace_tweens = [namespace for namespace in idx_tweens.list_namespaces()]
namespace_tweens

[{
     "name": "first_tokenization_thesis",
     "record_count": "116"
 }]